In [2]:
import numpy as np
import pandas as pd
import json
import functools

In [32]:
def outdated_data(func):
    @functools.wraps(func)  # сохраняет имя и docstring оригинальной функции
    def wrapper(self, *args, **kwargs):
        self.is_data_actual = False
        result = func(self, *args, **kwargs)
        return result
    return wrapper  # <-- ВАЖНО!

def uptodated_data(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        self.is_data_actual = True
        result = func(self, *args, **kwargs)
        return result
    return wrapper  # <-- ВАЖНО!


        
class Player:

    @uptodated_data
    def __init__(self, data: dict):
        must_data_fields = ['account_id', 'steamid', 'profileurl', 'personaname', 'cheese',
                            'fh_unavailable', 'loccountrycode', 'plus', 'name', 'country_code',
                            'fantasy_role', 'team_id', 'team_name', 'team_tag', 'is_locked',
                             'is_pro', 'locked_until']

        for field in must_data_fields:
            setattr(self, field, data.get(field))

    def __repr__(self):
        return f'Player {self.account_id} {self.profileurl} {self.personaname}\n'

    #@outdated_data
    #def add_match

    def __eq__(self, other):
        if not isinstance(other, Player):
            return NotImplemented
        return self.account_id == other.account_id

    def __hash__(self):
        return hash(self.account_id)

class Players:

    def __init__(self):
        self.players = []
        
    def add_player(self, player: Player):
        self.players.append(player)

    def __contains__(self, item):
        return item in self.players

    def __iter__(self):
        return iter(self.players)

    def __repr__(self):
        return str(self.players)

In [6]:
class Match:
    def __init__(self, data: dict):
        if data == '-1':
            raise ValueErorr('It is bad match')

        data['draft_timings'] = sorted(data['draft_timings'], key=lambda d: d['order'])

        self.match_id = data['match_id']
        
        simple_fields = ['match_id', 'barracks_status_dire', 'barracks_status_radiant', 'cluster',
                         'dire_score', 'duration', 'engine', 'first_blood_time', 'game_mode',
                         'human_players', 'leagueid', 'lobby_type', 'match_seq_num', 'negative_votes',
                         'positive_votes', 'radiant_score', 'radiant_win', 'start_time', 'tower_status_dire',
                         'tower_status_radiant', 'version', 'replay_salt', 'series_id', 'series_type',
                         'radiant_team', 'dire_team', 'league', 'skill', 'patch', 'region', 'throw',
                         'comeback', 'loss', 'win', 'replay_url']

        draft_fields = ['order', 'pick', 'active_team', 'hero_id', 'player_slot', 'extra_time', 'total_time_taken']
        
        players_info_fileds = ['player_slot', 'account_id', 'assists', 'camps_stacked', 'deaths', 'denies',
                               'gold', 'gold_per_min', 'gold_spent', 'hero_damage', 'hero_healing',
                               'hero_id', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'kills',
                               'last_hits', 'leaver_status', 'level', 'obs_placed', 'party_id', 'hero_variant',
                               'pings', 'rune_pickups', 'sen_placed', 'stuns', 'tower_damage', 'xp_per_min',
                               'personaname', 'name', 'last_login', 'isRadiant', 'total_gold', 'total_xp',
                               'kills_per_min', 'kda', 'abandons', 'neutral_kills', 'tower_kills', 'courier_kills',
                               'lane_kills', 'hero_kills', 'observer_kills', 'sentry_kills', 'roshan_kills',
                               'necronomicon_kills', 'ancient_kills', 'buyback_count', 'observer_uses', 'sentry_uses',
                               'lane_efficiency', 'lane_efficiency_pct', 'lane', 'lane_role', 'is_roaming',
                               'purchase_tpscroll', 'actions_per_min', 'life_state_dead', 'rank_tier']
        
        max_drafts = 25
        max_picks_bans = 25
        max_radiant_gold_adv = 180
        rad_gold_adv = 'radiant_gold_adv'
        max_radiant_xp_adv = 180
        rad_xp_adv = 'radiant_xp_adv'
        player_slots = 12

        match_info = dict()

        for i in range(max_radiant_gold_adv):
            match_info[f'{rad_gold_adv}_{i}'] = None

        for i in range(len(data[rad_gold_adv])):
            match_info[f'{rad_gold_adv}_{i}'] = data[rad_gold_adv][i]

        for i in range(max_radiant_xp_adv):
            match_info[f'{max_radiant_xp_adv}_{i}'] = None

        for i in range(len(data[rad_xp_adv])):
            match_info[f'{rad_xp_adv}_{i}'] = data[rad_xp_adv][i]


        for i in range(max_drafts):
            for field in draft_fields:
                match_info[f'draft_{field}_i'] = None

        for draft in data['draft_timings']:
            order = draft['order']
            for field in draft:
                match_info[f'draft_{field}_{order}'] = draft[field]


        for player_field in players_info_fileds:
            for i in range(player_slots):
                match_info[f'{player_field}_{i}'] = None

        
        for i in range(len(data['players'])):
            player = data['players'][i]
            for field in players_info_fileds:
                match_info[f'{field}_{i}'] = player[field]
        
        for field in simple_fields:
            if field in data:
                match_info[field] = data[field]

        self.match_info = match_info


    def get_data(self):
        return self.match_info
    

    def __eq__(self, other):
        if not isinstance(other, Match):
            return NotImplemented
        return self.account_id == other.account_id

    def __hash__(self):
        return hash(self.account_id)

    def __repr__(self):
        return str(self.__dict__)

In [34]:
def read_players(file='top_players.json'):
    with open(file, 'r', encoding='cp850') as file:
        players_raw_data = json.load(file)
        players = Players()
        for player_raw_data in players_raw_data:
            player = Player(player_raw_data)
            players.add_player(player)
            
    return players

In [7]:
with open(f'backup_7.json', 'r', encoding='cp850') as file:
    parsed_matches = json.load(file)
    pd_data = []
    for match in parsed_matches:
        if parsed_matches[match] == '-1' or parsed_matches[match]['game_mode'] != 22:
            continue
        match = Match(parsed_matches[match])
        pd_data.append(match.get_data())
    df = pd.DataFrame(pd_data)

In [82]:
df.to_csv('data_7.csv')

In [88]:
df['match_id']

0      7076993233
1      7076600098
2      7076531986
3      7076479096
4      7075707018
          ...    
268    6917731435
269    6917680848
270    6917631205
271    6916375861
272    6916290495
Name: match_id, Length: 273, dtype: int64

In [9]:
for match in parsed_matches:
    if 'radiant_team' in parsed_matches[match]:
        print(parsed_matches[match]['league'])
        exit()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
{'leagueid': 14921, 'ticket': None, 'banner': None, 'tier': 'premium', 'name': 'DPC 2023 WEU Winter Tour Division II ÔÇô presented by PGL'}
{'leagueid': 14921, 'ticket': None, 'banner': None, 'tier': 'premium', 'name': 'DPC 2023 WEU Winter Tour Division II ÔÇô presented by PGL'}
None
None
None
None
None
{'leagueid': 14921, 'ticket': None, 'banner': None, 'tier': 'premium', 'name': 'DPC 2023 WEU Winter Tour Division II ÔÇô presented by PGL'}
{'leagueid': 14921, 'ticket': None, 'banner': None, 'tier': 'premium', 'name': 'DPC 2023 WEU Winter Tour Division II ÔÇô presented by PGL'}
None


In [1]:
pd.read()

NameError: name 'pd' is not defined